# Understanding Higher-Kinded Types in Scala

Higher-kinded types are types of a higher kind. That might sound circular, but it really comes down to levels of abstraction. While generic types abstract over types, higher-kinded types abstract over generic types. Let's break this down.

## The 'Kind' of a Type

In Scala, we often talk about the 'kind' of a type to express its level of abstraction.
- The kind of `Int` is `*`, meaning it's a concrete type.
- The kind of `List` is `* -> *`, meaning it takes a concrete type and returns another concrete type.
- The kind of `Either` is `* -> * -> *`, meaning it takes two concrete types and returns a concrete type.

## Basic Example

Let's start with a simple (relatively speaking... none of this is particularly simple, but practice renders it far less mystical) example that demonstrates a method that abstracts over different container types.

Note that the instances provided here are type classes! We've already seen companion objects and the use of implicits to define type-safe equivalents of duck typing, so recall that in the wild it these implicit implementations would likely live inside their companion objects (`List` and `Option`, respectively).

In [ ]:
trait Functor[F[_]] {
  def map[A, B](fa: F[A])(f: A => B): F[B]
}

def transform[F[_], A, B](container: F[A], f: A => B)(implicit functor: Functor[F]): F[B] = {
  functor.map(container)(f)
}

// Functor instance for List
implicit object ListFunctor extends Functor[List] {
  def map[A, B](fa: List[A])(f: A => B): List[B] = fa.map(f)
}

// Functor instance for Option
implicit object OptionFunctor extends Functor[Option] {
  def map[A, B](fa: Option[A])(f: A => B): Option[B] = fa.map(f)
}

// Using ListFunctor
val list1 = List(1, 2, 3)
val list2 = transform(list1, (x: Int) => x + 1)

// Using OptionFunctor
val opt1: Option[Int] = Some(1)
val opt2 = transform(opt1, (x: Int) => x + 1)


Here, `Functor` is a higher-kinded type. It abstracts over types that themselves abstract over types.

> NOTE: We've included an explicit type (`Option[Int]`). This is because of a subtle and tricky difficulty in Scala's type inference. Because our `Functor` instance is defined over `Option`s, we need `opt1` to be recognized as an `Option` rather than one of its subtypes (in this case, `Some`). By telling the compiler that our `Some` should be upcast to the type of `Option`, we can skirt a compilation issue. Try removing the explicit type argument so that you can see how things go wrong.

## Why Higher-Kinded Types?

Higher-kinded types shine in their ability to write extremely generic code. They are often used in:
- Abstracting over various container types (`List`, `Option`, `Future`)
- Libraries that require a high level of code reuse
- Algebraic structures in functional programming like `Functors`, `Monads`, etc.

## Exercise

1. Create a higher-kinded type `Container` with a method `isEmpty` that checks whether the container is empty.
2. Implement `Container` for `List` and `Option`.